In [1]:
import numpy as np
import logging
logging.getLogger("pvtrace").setLevel(logging.CRITICAL)
logging.getLogger("trimesh").setLevel(logging.CRITICAL)
logging.getLogger("shapely.geos").setLevel(logging.CRITICAL)

In [2]:
from pvtrace import (
    Scene, Node,
    Sphere,
    Material, Surface, Ray,
    MeshcatRenderer, photon_tracer
)

# Example

Let's make a scene. 

Every scene must have a **world node** which contains all other objects. In the example below the world is a giant sphere (radius 10) filled with air (refractive index 1.0).

Inside the world node we add a smaller sphere (radius 1) which has refractive index 1.5 (this could represent glass or plastic).


## A simple scene

In [3]:
world = Node(
    name="world (air)",
    geometry=Sphere(
        radius=10.0,
        material=Material(refractive_index=1.0),
    )
)
sphere = Node(
    name="sphere (glass)",
    geometry=Sphere(
        radius=1.0,
        material=Material(refractive_index=1.5),
    ),
    parent=world
)
scene = Scene(world)

## Using the visualiser
Let's visualise this scene with the meshcat based renderer.

In [4]:
vis = MeshcatRenderer()
vis.render(scene)
vis.vis.jupyter_cell()

You can open the visualizer by visiting the following URL:
http://127.0.0.1:7000/static/


## Making rays
Throw some rays at the scene. A ray has a position, direction and wavelength.

In [5]:
ray = Ray(
    position=(0.0, 1.0, 1.1),
    direction=(0.0, -1.0, 0.0),
    wavelength=555.0
)

## Tracing the scene

Let's follow the ray through the scene using the `follow` function of the `photon_tracer` module. Drag the visualisation to see the path of the ray.

In [6]:
np.random.seed(0)
steps = photon_tracer.trace(scene, ray)
path, decisions = zip(*steps)
vis.add_ray_path(path)

The photon tracer simulates the propagation of the ray through the scene as if it was a photon. Automatic Fresnel reflection and refraction occurs at boundaries between materials for different refractive index. As you will see later, if the material also has an absopative or emissive properties rays can be absorbed and re-emitted.

The follow method returns a list of steps. At each step something happened. A step is just a tuple containing a ray and a decision enum.

In [7]:
steps[0]

(Ray(position=(0.00, 1.00, 1.10), direction=(0.00, -1.00, 0.00), wavelength=555.00, is_alive=True),
 <Event.GENERATE: 0>)

The first step is the creation of the ray with an `GENERATE` event.

In [8]:
steps[1]

(Ray(position=(0.00, -9.94, 1.10), direction=(0.00, -1.00, 0.00), wavelength=555.00, is_alive=True),
 <Event.EXIT: 6>)

The ray then hits the world node and exits. This is marked as the `EXIT` event.

Let's make a ray that intersects with the sphere and see what happens (you may need to scroll up to see the viewer).

In [9]:
ray = Ray(
    position=(0.0, -1.0, 0.9),
    direction=(0.0, 1.0, 0.0),
    wavelength=650.0
)
steps = photon_tracer.trace(scene, ray)
path, decisions = zip(*steps)
vis.add_ray_path(path)

This is a Monte Carlo simulation, the ray will not necessarily take the same path every time.

Let's generate 100 identical rays and see what happens.

In [10]:
import time
for _ in range(100):
    steps = photon_tracer.trace(scene, ray)
    path, decisions = zip(*steps)
    vis.add_ray_path(path)
    time.sleep(0.001)  # allow the renderer a chance to redraw

The other tutorial and example files show how to photon trace materials which also absorb and emit light and how to import complex geometies from mesh files and ray trace those.